In [ ]:
import random
import pickle
import pandas as pd
import spotipy
from random import randint
from IPython.display import clear_output

from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
secrets_file = open("spotkey.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [ ]:
# Import previously saved model
def import_component(path):
    return pickle.load(open(path, 'rb'))

In [ ]:
transformer = import_component('Model_data/scaler.sav')
model = import_component('Model_data/model_clustering.sav')

In [ ]:
df_top100 = pd.read_csv('Data/Final/Top100.csv')
df_clustered = pd.read_csv('Data/Final/Spotify dataset clustered.csv')

In [ ]:
def song_uri(song_id):
    try:
        # Creating the spotipy element for the playlist URI
        querry = 'track:'+str(song_id)
        track = sp.search(q=song_id, limit=1)
        return track['tracks']['items'][0]['uri'].split('spotify:track:')[1]                 
    except:
        return 'Null'

In [ ]:
def get_details(uri):
    # Creatinga a dataframe with the columns that we need
    playlist_lst = ['danceability','energy','key','loudness','mode', 'speechiness',
                    'instrumentalness','liveness','valence','tempo',
                    'duration_ms','time_signature']
    
    playlist_df = pd.DataFrame(columns = playlist_lst)
    # Get audio features and adding them to the respective columns
    audio_features = sp.audio_features(uri)[0]
    playlist_df.loc[len(playlist_df)] = [audio_features[i] for i in playlist_lst]
    return playlist_df

In [ ]:
while True:
    
    # User inserts search term
    print('Insert song or search term:')
    search=input()

    # flag that controls if the search string exists in the songs dataset
    exist = False

    # testing if the search string exists
    for i in ['song', 'artist', 'genre']:
        if len(df_top100[df_top100[i].str.contains(search, case = False, regex = False)]) != 0:
            exist = True
    # if the song or search term exists in the dataset we sugest a random song, otherwise try tomorrow
    if exist == True:
        clear_output()
        index = random.randint(0,len(df_top100))
        print('\nI have a sugestion! \n\nLink: ','https://open.spotify.com/track/'+df_top100['uri'].values[index],  '\nSong: ',df_top100['song'].values[index],  '\nArtist: ', df_top100['artist'].values[index])      
    else:
        try:
            clear_output()
            df = get_details(song_uri(search))
            cluster = model.predict(pd.DataFrame(transformer.transform(df), columns = df.columns))
            print('Spotify sugestion!')
            element = df_clustered[df_clustered['cluster'] == int(cluster)]
            index = random.randint(0,len(element))
            print('\n\nLink: ','https://open.spotify.com/track/'+element['track_id'].values[index],  '\nSong: ',element['track_name'].values[index],  '\nArtist: ', element['artist'].values[index])
        except:
            print('Invalid song. Sorry!')


Invalid song. Sorry!
Insert song or search term:
